# T-test
example

In [67]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
from collections import Counter

In [ ]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [2]:
np.random.seed(99)

# Create sample data.
a = np.random.randn(10000)
b = np.random.randn(10000) + 0.1

# Use scipy.stats.ttest_ind.
t, p = ttest_ind(a, b, equal_var=False)

print('ttest_ind:' + str(p))

ttest_ind:2.81708330972e-07


# Cell-Type Populations in Plasma and PMA Treated

## Load Data

In [3]:
# Plasma Treated
net.load_file('../cytof_data/Plasma_UCT.txt')
net.random_sample(axis='row', num_samples=110000, random_state=99)
df_plasma = net.export_df()

print(df_plasma.shape)

# PMA Treated
net.load_file('../cytof_data/PMA_UCT.txt')
net.random_sample(axis='row', num_samples=110000, random_state=99)
df_pma = net.export_df()

print(df_pma.shape)

(110000, 28)
(110000, 28)


## Randomly Sample Data and Calc Cell-Type Populations

In [122]:
np.random.seed(99)
num_samples = 2000
num_runs = 100

sampled_counts = {}
sampled_counts['plasma'] = {}
sampled_counts['pma'] = {}

for inst_treatment in ['plasma', 'pma']:
    
    for i in range(num_runs):

        # randomly subsample 
        random_state = int(np.random.random()*100)    
        net.load_df(df_plasma)
        net.random_sample(axis='row', num_samples=num_samples, random_state=random_state)
        inst_df = net.export_df()
        inst_rows = inst_df.index.tolist()

        # get cell-types from rows
        all_types = [i[2] for i in inst_rows]

        types_list = sorted(list(set(all_types)))

        # get the counts of all cell types in the subsampled data
        inst_counts = Counter(all_types)

        for inst_type in inst_counts.keys():

            # initialize counts
            if inst_type not in sampled_counts[inst_treatment]:
                sampled_counts[inst_treatment][inst_type] = []

            # append new count
            sampled_counts[inst_treatment][inst_type].append(inst_counts[inst_type])
            
    # change to numpy array 
    for inst_type in sampled_counts[inst_treatment]:
        sampled_counts[inst_treatment][inst_type] = np.asarray(sampled_counts[inst_treatment][inst_type])
    
    
    
print('done')

done


In [124]:
print(sampled_counts['pma']['B cells'].mean())
print(sampled_counts['pma']['B cells'].std())

124.3
9.45991543303


In [125]:
print(sampled_counts['plasma']['B cells'].mean())
print(sampled_counts['plasma']['B cells'].std())

124.31
11.3821746604


## Calc Stats and Significant Differences Between Plasma and PMA

In [126]:
results = {}

for inst_type in sampled_counts['plasma']:
    
    results[inst_type] = {}
    
    a = sampled_counts['plasma'][inst_type]
    b = sampled_counts['pma'][inst_type]

    a_mean = a.mean()
    b_mean = b.mean()
    
    a_std = a.std()
    b_std = b.std()    
    
    results[inst_type]['plasma_mean'] = a_mean
    results[inst_type]['pma_mean'] = b_mean   

    results[inst_type]['plasma_std'] = a_std
    results[inst_type]['pma_std'] = b_std      
    
    print(inst_type)
    print('Plasma: ' + str(a_mean) + ' PMA: ' + str(b_mean))

    t, p = ttest_ind(a, b, equal_var=False)
    
    results[inst_type]['ttest_t'] = t
    results[inst_type]['ttest_pval'] = t

    print('ttest_ind: ' + str(p) + '\n')

NK cells_CD56hi
Plasma: 21.7 PMA: 22.17
ttest_ind: 0.427762153249

CD4 Tcells
Plasma: 472.61 PMA: 473.67
ttest_ind: 0.696218828586

Undefined
Plasma: 78.24 PMA: 77.55
ttest_ind: 0.528964347678

B cells
Plasma: 124.31 PMA: 124.3
ttest_ind: 0.994642994464

CD8 Tcells
Plasma: 277.63 PMA: 280.31
ttest_ind: 0.271429925343

CD1c DCs
Plasma: 46.42 PMA: 47.06
ttest_ind: 0.468327298808

NK cells_CD16hi_CD57hi
Plasma: 119.27 PMA: 117.07
ttest_ind: 0.108796687434

CD4 Tcells_CD161hi
Plasma: 72.15 PMA: 72.14
ttest_ind: 0.993150834628

NK cells_CD16hi
Plasma: 156.52 PMA: 157.14
ttest_ind: 0.728304150694

CD14hi monocytes
Plasma: 170.72 PMA: 168.98
ttest_ind: 0.333920750767

CD4 Tcells+CD27hi
Plasma: 218.17 PMA: 218.19
ttest_ind: 0.991280263886

CD14low monocytes
Plasma: 10.89 PMA: 11.28
ttest_ind: 0.475338973344

CD4 Tcells_CD127hi
Plasma: 86.55 PMA: 86.95
ttest_ind: 0.779181874194

Basophils
Plasma: 24.13 PMA: 23.06
ttest_ind: 0.124360203148

Neutrophils
Plasma: 2.88172043011 PMA: 2.69411764706
tt

In [127]:
results

{'B cells': {'plasma_mean': 124.31,
  'plasma_std': 11.382174660406507,
  'pma_mean': 124.3,
  'pma_std': 9.4599154330258166,
  'ttest_pval': 0.0067228282389601808,
  'ttest_t': 0.0067228282389601808},
 'Basophils': {'plasma_mean': 24.129999999999999,
  'plasma_std': 4.8013643894209901,
  'pma_mean': 23.059999999999999,
  'pma_std': 4.9534230588553605,
  'ttest_pval': 1.5432838236493933,
  'ttest_t': 1.5432838236493933},
 'CD14hi monocytes': {'plasma_mean': 170.72,
  'plasma_std': 12.071520202526274,
  'pma_mean': 168.97999999999999,
  'pma_std': 13.181031826074921,
  'ttest_pval': 0.96862971125329356,
  'ttest_t': 0.96862971125329356},
 'CD14low monocytes': {'plasma_mean': 10.890000000000001,
  'plasma_std': 3.7970909917988536,
  'pma_mean': 11.279999999999999,
  'pma_std': 3.8757708910615447,
  'ttest_pval': -0.71518268187145695,
  'ttest_t': -0.71518268187145695},
 'CD1c DCs': {'plasma_mean': 46.420000000000002,
  'plasma_std': 5.7011928576395308,
  'pma_mean': 47.060000000000002,
 

In [128]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=['Trial 1', 'Trial 2', 'Trial 3'],
    y=[3, 6, 4],
    name='Control',
    error_y=dict(
        type='data',
        array=[1, 0.5, 1.5],
        visible=True
    )
)
trace2 = go.Bar(
    x=['Trial 1', 'Trial 2', 'Trial 3'],
    y=[4, 7, 3],
    name='Experimental',
    error_y=dict(
        type='data',
        array=[0.5, 1, 2],
        visible=True
    )
)
data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='error-bar-bar')


ImportError: No module named plotly.plotly